In [1]:
import numpy as np
import pandas as pd
import glob
import os

### Paper data

In [5]:
input = np.load("/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Preprocessed_external_data_1/Running/SUB01/LeftFoot/acc.npy", allow_pickle=True).astype(float)
print(input.shape)
print(input)

(142, 3)
[[ -9.1643613  -9.2936232  13.1300684]
 [-10.3970888 -11.7245562  13.8706617]
 [-12.4244112 -14.7736683  15.6709578]
 [-15.6728994 -18.0266035  18.8326045]
 [-20.5905982 -22.8064721  23.0351481]
 [-22.3753664 -27.0142686  26.0540526]
 [-20.1684111 -27.9764761  26.9990636]
 [-17.1361096 -23.6307773  25.762182 ]
 [-13.228142  -17.7892763  24.2555128]
 [ -9.7666133 -13.1374922  22.7225394]
 [ -7.144621   -9.1255428  21.1002362]
 [ -6.9889748  -5.4453381  19.5590276]
 [ -6.7351045  -3.9336779  17.0123685]
 [ -6.2791798  -4.4983164  14.1442525]
 [ -7.4027994  -3.4777879  11.5354599]
 [-10.6133938  -2.4751268   9.3297572]
 [-14.0011541  -1.1967052   8.5598983]
 [-16.2443291   0.2180938   9.0313931]
 [-16.9813553  -0.4900125   9.22417  ]
 [-16.6854027  -2.4554374   9.7672045]
 [-15.7845631  -4.5041742  11.2194952]
 [-15.3446951  -5.9955625  13.4168495]
 [-16.5692806  -7.6309416  16.6063819]
 [-19.2779345  -9.0929992  20.6970938]
 [-21.4148228  -9.1424952  23.7085873]
 [-23.6815914  -

In [6]:
label = np.load("/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/s999/lankle/angle.npy")
print(label.shape)
print(label)

(4600, 3)
[[  5.09249264  -7.5996024  -12.61360109]
 [  4.72438656  -7.57943448 -12.25711487]
 [  4.37830979  -7.54552855 -11.8421057 ]
 ...
 [-10.0993935   -4.56046515  -0.82373543]
 [-10.25273302  -4.61637978  -0.92882851]
 [-10.40019937  -4.68304039  -1.03708614]]


### External data preprocessing (Dataset 3)

In [6]:
Target_merge_path = "/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Data/IMUExerciseClassification/parsed_joint_angles/"

In [ ]:
def merge_csv_files(folder_path):
    # Get all subdirectories (SUB folders)
    sub_dirs = sorted(glob.glob(f"{folder_path}/*/"))
    
    for sub_dir in sub_dirs:
        for category in ["Walk", "Run"]:
            category_path = os.path.join(sub_dir, category)
            file_paths = sorted(glob.glob(f"{category_path}/*.csv"))
            
            if not file_paths:
                print(f"No file in: {category_path}")
                continue
            
            '''Merge all files'''
            # Read the first file and manually retain the first row (body part) and header
            main_df = pd.read_csv(file_paths[0], header=None, keep_default_na=False)
            # Read the remaining files (without headers, ensuring column alignment)
            dfs = [pd.read_csv(file, header=None, names=main_df.columns, skiprows=3, keep_default_na=False) for file in file_paths[1:]]
            # Merge all DataFrames
            merged_df = pd.concat([main_df] + dfs, ignore_index=True)
            
            '''Select and rename columns''' 
            
            '''Save merged data'''
            # Define output file path
            output_file = os.path.join(category_path, f"Merged_{category}.csv")
            
            # Save the merged DataFrame as a new CSV file
            merged_df.to_csv(output_file, index=False, header=True)
            print(f"Merge completed, saved as: {output_file}")
            
merge_csv_files(Target_merge_path)

Merge completed, saved as: /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Data/IMUExerciseClassification/parsed_joint_angles/SUB01/Walk/Merged_Walk.csv
Merge completed, saved as: /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Data/IMUExerciseClassification/parsed_joint_angles/SUB01/Run/Merged_Run.csv
Merge completed, saved as: /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Data/IMUExerciseClassification/parsed_joint_angles/SUB02/Walk/Merged_Walk.csv
Merge completed, saved as: /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Data/IMUExerciseClassification/parsed_joint_angles/SUB02/Run/Merged_Run.csv
Merge completed, saved as: /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Data/IMUExerciseClassification/parsed_joint_angles/SUB03/Walk/Merged_Walk.csv
Merge completed, saved as: /Users/

In [ ]:
import pandas as pd
# Load the CSV file
file_path = "/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Data/IMUExerciseClassification/parsed_h5_csv/SUB01/Run/Merged_Run.csv"
df = pd.read_csv(file_path)
# Display the first few rows to understand its structure
df.head()

,0,1,2,3,4,5,6,7,8,9,...,131,132,133,134,135,136,137,138,139,140
0,NaN,Chest,Chest,Chest,Chest,Chest,Chest,Chest,Chest,Chest,...,RightWrist,RightWrist,RightWrist,RightWrist,RightWrist,RightWrist,RightWrist,RightWrist,RightWrist,RightWrist
1,NaN,Time,Orientation,Orientation,Orientation,Orientation,Accelerometer,Accelerometer,Accelerometer,Gyroscope,...,Orientation,Accelerometer,Accelerometer,Accelerometer,Gyroscope,Gyroscope,Gyroscope,Magnetometer,Magnetometer,Magnetometer
2,NaN,S,S,X,Y,Z,X,Y,Z,X,...,Z,X,Y,Z,X,Y,Z,X,Y,Z
3,0.0,7.8500000,0.1192912,-0.7097131,0.1786335,0.6709449,-2.0051832,-1.6421057,-2.4460219,-1.0478124,...,0.8311360,10.3023797,1.2534952,4.3392142,-0.0228866,-1.3629788,1.1179846,27.3779582,-32.7676157,-18.7968226
4,1.0,7.8600000,0.1140533,-0.7077450,0.1786573,0.6739216,-1.8338254,-2.3406476,-1.5110177,-0.9736381,...,0.8345135,9.2173085,1.7636103,3.6514502,-0.1313005,-1.2365006,1.3051321,26.7301199,-32.7562691,-19.0897938


### Create folder

In [6]:
processed_data_path = "/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Preprocessed_external_data_1"

In [10]:
patients = ['SUB01', 'SUB02', 'SUB03', 'SUB04', 'SUB05', 'SUB06', 'SUB07', 'SUB08', 'SUB09', 
            'SUB10', 'SUB11', 'SUB12', 'SUB13', 'SUB14','SUB15', 'SUB16', 'SUB17', 'SUB18', 'SUB19']
motions = ['Walk', 'Run']
# Define body parts to extract
body_parts = ["Pelvis", "LeftThigh", "RightThigh", "LeftShank", "RightShank", "LeftFoot", "RightFoot"]
angle_parts = ["LeftAnkleAngle", "RightAnkleAngle", "LeftHipAngle", "RightHipAngle", "LeftKneeAngle", "RightKneeAngle"]
for patient in patients:
    for motion in motions:
        for part in body_parts:
            os.makedirs(os.path.join(processed_data_path, patient, motion, part), exist_ok=True)

### Generate npy files

In [8]:
folder_path = "/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/wearable-project/data preprocessing/Data/IMUExerciseClassification/parsed_joint_angles"

In [10]:
def generate_angle_npy(folder_path):
    # Load the CSV file
    # Get all subdirectories (SUB folders)
    sub_dirs = sorted(glob.glob(f"{folder_path}/*/"))
    
    for sub_dir in sub_dirs:
        patient_number = os.path.basename(os.path.normpath(sub_dir))
        for category in ["Walk", "Run"]:
            category_path = os.path.join(sub_dir, category)
            file_paths = sorted(glob.glob(f"{category_path}/Merged*.csv"))
            df = pd.read_csv(file_paths[0], header=None)
            
            '''Select columns and rename'''
            # Step 1: Extract headers from the first three rows
            body_parts_row = df.iloc[1]  # Body part names
            sensor_type_row = df.iloc[2]  # Sensor type (Accelerometer, Gyroscope, etc.)
            axis_row = df.iloc[3]  # X, Y, Z

            # Create new column names by combining these rows
            new_column_names = [
                f"{body_part}_{sensor}_{axis}" 
                for body_part, sensor, axis in zip(body_parts_row, sensor_type_row, axis_row)
            ]
            
            # Step 2: Create a new DataFrame with correct headers
            df_cleaned = df.iloc[3:].reset_index(drop=True)  # Keep only numeric data
            df_cleaned.columns = new_column_names  # Assign meaningful column names
            
            # Define body parts to extract
            angle_parts = ["LeftAnkleAngle", "RightAnkleAngle", "LeftHipAngle", "RightHipAngle", "LeftKneeAngle", "RightKneeAngle"]
            for part in angle_parts:
                # Extract Accelerometer and Gyroscope data for X, Y, Z
                angles_columns = [col for col in df_cleaned.columns if f"{part}_ML" in col]
                # Ensure correct order of X, Y, Z
                angles_columns_sorted = sorted(angles_columns, key=lambda x: ['X', 'Y', 'Z'].index(x.split('_')[-1]))
                # Extract data and ensure shape (n,3)
                angles_data = df_cleaned[angles_columns_sorted].to_numpy() if angles_columns_sorted else None
                
                # Save as .npy files if valid data exists
                if angles_data is not None and angles_data.shape[1] == 3:
                    np.save(os.path.join(processed_data_path, category,patient_number, part, "angle.npy"), angles_data[1:])
                    
generate_angle_npy(folder_path)

In [13]:
folder_path = "/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Data/IMUExerciseClassification/parsed_h5_csv/"

In [14]:
def generate_imu_npy(folder_path):
    # Load the CSV file
    # Get all subdirectories (SUB folders)
    sub_dirs = sorted(glob.glob(f"{folder_path}/*/"))
    
    for sub_dir in sub_dirs:
        patient_number = os.path.basename(os.path.normpath(sub_dir))
        for category in ["Walk", "Run"]:
            category_path = os.path.join(sub_dir, category)
            file_paths = sorted(glob.glob(f"{category_path}/Merged*.csv"))
            df = pd.read_csv(file_paths[0], header=None)
            
            '''Select columns and rename'''
            # Step 1: Extract headers from the first three rows
            body_parts_row = df.iloc[1]  # Body part names
            sensor_type_row = df.iloc[2]  # Sensor type (Accelerometer, Gyroscope, etc.)
            axis_row = df.iloc[3]  # X, Y, Z

            # Create new column names by combining these rows
            new_column_names = [
                f"{body_part}_{sensor}_{axis}" 
                for body_part, sensor, axis in zip(body_parts_row, sensor_type_row, axis_row)
            ]
            
            # Step 2: Create a new DataFrame with correct headers
            df_cleaned = df.iloc[3:].reset_index(drop=True)  # Keep only numeric data
            df_cleaned.columns = new_column_names  # Assign meaningful column names
            # Define body parts to extract
            body_parts = ["Pelvis", "LeftThigh", "RightThigh", "LeftShank", "RightShank", "LeftFoot", "RightFoot"]
            for part in body_parts:
                # Extract Accelerometer and Gyroscope data for X, Y, Z
                acc_columns = [col for col in df_cleaned.columns if f"{part}_Accelerometer" in col]
                gyr_columns = [col for col in df_cleaned.columns if f"{part}_Gyroscope" in col]
                # Ensure correct order of X, Y, Z
                acc_columns_sorted = sorted(acc_columns, key=lambda x: ['X', 'Y', 'Z'].index(x.split('_')[-1]))
                gyr_columns_sorted = sorted(gyr_columns, key=lambda x: ['X', 'Y', 'Z'].index(x.split('_')[-1]))
                # Extract data and ensure shape (n,3)
                acc_data = df_cleaned[acc_columns_sorted].to_numpy() if acc_columns_sorted else None
                gyr_data = df_cleaned[gyr_columns_sorted].to_numpy() if gyr_columns_sorted else None
                # Save as .npy files if valid data exists
                if acc_data is not None and acc_data.shape[1] == 3:
                    np.save(os.path.join(processed_data_path, patient_number, category, part, "acc.npy"), acc_data[1:])
                
                if gyr_data is not None and gyr_data.shape[1] == 3:
                    np.save(os.path.join(processed_data_path, patient_number, category, part, "gyr.npy"), gyr_data[1:])
                        
generate_imu_npy(folder_path)

### Convert .npy into .h5

In [ ]:
import h5py

npy_directory = "/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Preprocessed_data/"  # Change this to your npy folder
def convert_to_h5(folder_path):
# Define paths
    h5_filename = "Data.h5"
    # Create an HDF5 file
    with h5py.File(h5_filename, 'w') as h5f:
        for root, _, files in os.walk(folder_path):
            for file in files:
                if file.endswith(".npy"):
                    file_path = os.path.join(root, file)
                    print(file_path)
                    data = np.load(file_path, allow_pickle=True)  # Load .npy file

                    # Create a dataset name based on the file structure
                    dataset_name = os.path.relpath(file_path, folder_path).replace(os.sep, "/").replace(".npy", "")

                    # Store in HDF5
                    h5f.create_dataset(dataset_name, data=data)

                    print(f"Saved {file_path} as {dataset_name} in {h5_filename}")

    print(f"Conversion complete! Data saved in {h5_filename}")

/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Preprocessed_data/Walk/SUB11/LeftShank/gyr.npy
Saved /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Preprocessed_data/Walk/SUB11/LeftShank/gyr.npy as Walk/SUB11/LeftShank/gyr in Data.h5
/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Preprocessed_data/Walk/SUB11/LeftShank/acc.npy
Saved /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Preprocessed_data/Walk/SUB11/LeftShank/acc.npy as Walk/SUB11/LeftShank/acc in Data.h5
/Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Preprocessed_data/Walk/SUB11/RightHipAngle/angle.npy
Saved /Users/ccy/Documents/CMU/Spring2025/42696 Wearable Health Technology/Projects/Milestone2/Preprocessed_data/Walk/SUB11/RightHipAngle/angle.npy as Walk/SUB11/RightHipAngle/angle in Data.h5
/Users/ccy/Documents/CMU/Spring

### Rotate axis

In [ ]:
import numpy as np

def rotate_imu(data, axis, angle):
    """
    Rotate imu direction
    
    Parameters:
    - data: numpy array, (n, 3) -> X, Y, Z 
    - axis: str（'x', 'y', 'z'）
    - angle: int，(90,180,270)

    Returns:
    - numpy array
    """
    angle_rad = np.deg2rad(angle)  
    
    if axis == 'x':
        rotation_matrix = np.array([
            [1, 0, 0],
            [0, np.cos(angle_rad), -np.sin(angle_rad)],
            [0, np.sin(angle_rad), np.cos(angle_rad)]
        ])
    elif axis == 'y':
        rotation_matrix = np.array([
            [np.cos(angle_rad), 0, np.sin(angle_rad)],
            [0, 1, 0],
            [-np.sin(angle_rad), 0, np.cos(angle_rad)]
        ])
    elif axis == 'z':
        rotation_matrix = np.array([
            [np.cos(angle_rad), -np.sin(angle_rad), 0],
            [np.sin(angle_rad), np.cos(angle_rad), 0],
            [0, 0, 1]
        ])
    else:
        raise ValueError("axis should be 'x', 'y', or 'z'")

    rotated_data = data @ rotation_matrix.T
    return rotated_data
